In [1]:
from dao.mongodbdao import MongoDBDao
from updater.TF_IDF import TF_IDF
import numpy as np 


dao = MongoDBDao({
    "host": "127.0.0.1",
    "db": "default",
    "wordindex_collection": "wordindex",
    "pagedetails_collection": "pagedetails",
    "username": "root",
    "password": "spezanw"
})

In [2]:
tfidf = TF_IDF(dao)
tfidf.calcTF_IDFs()

TF-IDF calculation: article contains no words
TF-IDF calculation: article contains no words
TF-IDF calculation: article contains no words
TF-IDF calculation: article contains no words


In [3]:
tfidf.getResultsForInput(user_input=["fußgängerin", "gestern", "nachmittag", "polizei", "waffe", "drogen", "fahrrad"], n_results=10, return_sims=True)




[('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.329268.php',
  0.36379878995446746),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.465558.php',
  0.2950130924496261),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.342413.php',
  0.2761680111867712),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.256232.php',
  0.24272444660353934),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.694020.php',
  0.2280660674830215),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.243475.php',
  0.22662272369402217),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.564097.php',
  0.2169946844059683),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.734720.php',
  0.2145168862650541),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.387122.php',
  0.21446152164563945),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.53

TF: 
- für jedes Dokument alle Wörter

IDF:
- Anzahl der Dokumente
- alle Wörter
- in wie vielen Dokumenten kommt jedes Wort vor?

1. Index: Anzahl der Dokumente
2. Inverser Index: Für term t get t.length